In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Projects/mwp_enhanced/modeling2
# Install and Load Libraries

KeyboardInterrupt: ignored

# Install and Load Libraries

1.   List item
2.   List item



In [ ]:
%%capture captured
!pip install yahoo-fin
!pip install hvplot
!pip install pandas-ta
# url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
# !curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
# url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py39hd257fcd_4.tar.bz2'
# !curl -L $url | tar xj -C /usr/local/lib/python3.9/dist-packages/ lib/python3.9/site-packages/talib --strip-components=3
!pip install pycaret
!pip install "schemdraw<0.16"
!pip install mlflow --quiet
!pip install mplfinance
!pip install python-dotenv
# import modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import panel as pn
pn.extension('tabulator')
import pandas as pd

from panel.template import FastListTemplate
from pathlib import Path
from yahoo_fin.stock_info import get_data
import datetime
from matplotlib.figure import Figure
from matplotlib import cm
%matplotlib inline

# import modules that help build tabs
import modules.helpers as helpers
import modules.HistoricalData as hst
import modules.MCTab as MCTab
import modules.intro as intro
import modules.profile as prf
import modules.AlgoTab as at

import pandas_ta as ta
# import talib
# from yahoo_finance_api2 import share
import numpy as np
# import mplfinance as mpf

from joblib import dump, load
from pycaret.classification import *

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import mlflow
from getpass import getpass
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv

from sklearn.metrics import brier_score_loss


load_dotenv()


# Load datasets

In [ ]:
X_train = pd.read_csv(Path("./data/prepared/Xtrain.csv"))
X_test = pd.read_csv(Path("./data/prepared/Xtest.csv"))
y_train = pd.read_csv(Path("./data/prepared/ytrain.csv"))
y_test = pd.read_csv(Path("./data/prepared/ytest.csv"))
X_validation = pd.read_csv(Path("./data/prepared/Xvalidation.csv"))
y_validation = pd.read_csv(Path("./data/prepared/yvalidation.csv"))

In [ ]:
mlflow.set_tracking_uri(f"https://dagshub.com/tlchampion/map_trials.mlflow")


# Optimize Gradient Boosted Classifier

In [ ]:
gbc = ClassificationExperiment()
gbc.setup(X_train ,
             target = y_train.squeeze(),
             session_id = 123, 
             fix_imbalance=False, 
             normalize=True, 
             normalize_method='minmax',
             use_gpu = False,
             log_experiment = True,
             experiment_name = "gbc_tuning",
             log_plots = True)
gbc.add_metric("brier", "Brier Score", brier_score_loss, greater_is_better = False, target="pred_proba")

In [ ]:
gbc_model = gbc.create_model('gbc')

In [ ]:
gbc_tuned, tuner  = gbc.tune_model(gbc_model, return_tuner=True)